In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4706087965441977017, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5659663521025111709
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 8
img_rows, img_cols = 512,512
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [12]:
"""
https://github.com/lyakaap/Kaggle-Carvana-3rd-place-solution
model.compile(optimizer=RMSprop(lr), loss=bce_dice_loss, metrics=[dice_coef])
"""
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Input, add, concatenate
from keras.models import Model


def DilatedUnet(
        input_shape=(512, 512, 3),
        mode='cascade',
        filters=44,
        n_block=3,
        lr=0.0001,
        n_class=1
):
    inputs = Input(input_shape)

    enc, skip = encoder(inputs, filters, n_block)
    bottle = bottleneck(enc, filters_bottleneck=filters * 2**n_block, mode=mode)
    dec = decoder(bottle, skip, filters, n_block)
    classify = Conv2D(n_class, (1, 1), activation='sigmoid')(dec)

    model = Model(inputs=inputs, outputs=classify)

    return model


def encoder(x, filters=44, n_block=3, kernel_size=(3, 3), activation='relu'):
    skip = []
    for i in range(n_block):
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        skip.append(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    return x, skip


def bottleneck(x, filters_bottleneck, mode='cascade', depth=6,
               kernel_size=(3, 3), activation='relu'):
    dilated_layers = []
    if mode == 'cascade':  # used in the competition
        for i in range(depth):
            x = Conv2D(filters_bottleneck, kernel_size,
                       activation=activation, padding='same', dilation_rate=2**i)(x)
            dilated_layers.append(x)
        return add(dilated_layers)
    elif mode == 'parallel':  # Like "Atrous Spatial Pyramid Pooling"
        for i in range(depth):
            dilated_layers.append(
                Conv2D(filters_bottleneck, kernel_size,
                       activation=activation, padding='same', dilation_rate=2**i)(x)
            )
        return add(dilated_layers)


def decoder(x, skip, filters, n_block=3, kernel_size=(3, 3), activation='relu'):
    for i in reversed(range(n_block)):
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = concatenate([skip[i], x])
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
    return x

In [13]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [14]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [15]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 3264
predict_size_validation: 1088
predict_size_test: 4


In [16]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [17]:
model = DilatedUnet()
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 44) 1232        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 44) 17468       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 44) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [18]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
3264/3264 [==============================] - 2658s 814ms/step - loss: -0.3538 - dice_coef: 0.3538 - acc: 0.7562 - mean_squared_error: 0.0873 - Jaccard_index: 0.2250 - Specificity: 0.9652 - Sensitivity: 1.0702 - precision: 0.3567 - recall: 0.4874 - f1score: nan - val_loss: -0.5120 - val_dice_coef: 0.5120 - val_acc: 0.8144 - val_mean_squared_error: 0.0398 - val_Jaccard_index: 0.3501 - val_Specificity: 0.9562 - val_Sensitivity: 0.9986 - val_precision: 0.6786 - val_recall: 0.4717 - val_f1score: 0.5471
Epoch 2/50
3264/3264 [==============================] - 2641s 809ms/step - loss: -0.6306 - dice_coef: 0.6306 - acc: 0.8250 - mean_squared_error: 0.0266 - Jaccard_index: 0.4682 - Specificity: 0.9684 - Sensitivity: 1.0056 - precision: 0.7323 - recall: 0.6469 - f1score: 0.6771 - val_loss: -0.6436 - val_dice_coef: 0.6436 - val_acc: 0.8194 - val_mean_squared_error: 0.0299 - val_Jaccard_index: 0.4833 - val_Specificity: 0.9608 - val_Sensitivity: 1.0021 - val_precision: 0.7692 - val_recall

Epoch 17/50
3264/3264 [==============================] - 2659s 815ms/step - loss: -0.9010 - dice_coef: 0.9010 - acc: 0.8395 - mean_squared_error: 0.0037 - Jaccard_index: 0.8215 - Specificity: 0.9755 - Sensitivity: 1.0028 - precision: 0.9802 - recall: 0.9702 - f1score: 0.9750 - val_loss: -0.7748 - val_dice_coef: 0.7748 - val_acc: 0.8273 - val_mean_squared_error: 0.0175 - val_Jaccard_index: 0.6417 - val_Specificity: 0.9645 - val_Sensitivity: 0.9999 - val_precision: 0.9077 - val_recall: 0.7876 - val_f1score: 0.8351
Epoch 18/50
3264/3264 [==============================] - 2658s 814ms/step - loss: -0.9021 - dice_coef: 0.9021 - acc: 0.8396 - mean_squared_error: 0.0035 - Jaccard_index: 0.8234 - Specificity: 0.9755 - Sensitivity: 1.0027 - precision: 0.9813 - recall: 0.9715 - f1score: 0.9763 - val_loss: -0.7697 - val_dice_coef: 0.7697 - val_acc: 0.8280 - val_mean_squared_error: 0.0174 - val_Jaccard_index: 0.6360 - val_Specificity: 0.9637 - val_Sensitivity: 0.9974 - val_precision: 0.9329 - val_r

Epoch 33/50
3264/3264 [==============================] - 2629s 806ms/step - loss: -0.9100 - dice_coef: 0.9100 - acc: 0.8400 - mean_squared_error: 0.0029 - Jaccard_index: 0.8366 - Specificity: 0.9757 - Sensitivity: 1.0027 - precision: 0.9882 - recall: 0.9819 - f1score: 0.9850 - val_loss: -0.7726 - val_dice_coef: 0.7726 - val_acc: 0.8281 - val_mean_squared_error: 0.0172 - val_Jaccard_index: 0.6402 - val_Specificity: 0.9638 - val_Sensitivity: 0.9976 - val_precision: 0.9331 - val_recall: 0.7657 - val_f1score: 0.8318
Epoch 34/50
3264/3264 [==============================] - 2623s 804ms/step - loss: -0.9101 - dice_coef: 0.9101 - acc: 0.8400 - mean_squared_error: 0.0029 - Jaccard_index: 0.8370 - Specificity: 0.9757 - Sensitivity: 1.0027 - precision: 0.9884 - recall: 0.9824 - f1score: 0.9854 - val_loss: -0.7667 - val_dice_coef: 0.7667 - val_acc: 0.8280 - val_mean_squared_error: 0.0177 - val_Jaccard_index: 0.6316 - val_Specificity: 0.9635 - val_Sensitivity: 0.9971 - val_precision: 0.9351 - val_r

Epoch 49/50
3264/3264 [==============================] - 2639s 808ms/step - loss: -0.9129 - dice_coef: 0.9129 - acc: 0.8401 - mean_squared_error: 0.0026 - Jaccard_index: 0.8415 - Specificity: 0.9758 - Sensitivity: 1.0027 - precision: 0.9906 - recall: 0.9860 - f1score: 0.9883 - val_loss: -0.7689 - val_dice_coef: 0.7689 - val_acc: 0.8282 - val_mean_squared_error: 0.0175 - val_Jaccard_index: 0.6350 - val_Specificity: 0.9635 - val_Sensitivity: 0.9968 - val_precision: 0.9396 - val_recall: 0.7554 - val_f1score: 0.8279
Epoch 50/50
3264/3264 [==============================] - 2639s 809ms/step - loss: -0.9127 - dice_coef: 0.9127 - acc: 0.8401 - mean_squared_error: 0.0026 - Jaccard_index: 0.8413 - Specificity: 0.9758 - Sensitivity: 1.0027 - precision: 0.9907 - recall: 0.9861 - f1score: 0.9884 - val_loss: -0.7764 - val_dice_coef: 0.7764 - val_acc: 0.8279 - val_mean_squared_error: 0.0172 - val_Jaccard_index: 0.6444 - val_Specificity: 0.9642 - val_Sensitivity: 0.9987 - val_precision: 0.9236 - val_r

In [19]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  -0.7849679028987885
Validation Accuracy:  0.7849679028987885


In [20]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

28/28 [==============================] - 7s 246ms/step


In [21]:
prediction.shape

(196, 512, 512, 1)

In [22]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

['loss', 'dice_coef', 'acc', 'mean_squared_error', 'Jaccard_index', 'Specificity', 'Sensitivity', 'precision', 'recall', 'f1score']
[-0.8210394735308899, 0.8210394735308899, 0.9837139611956717, 0.016248789931722414, 0.7092016657878613, 0.9699039459228516, 0.999576981040253, 0.8749787334738106, 0.7907110562269715, 0.8210282692278938]


In [ ]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()